In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('houseprice.csv')
test = pd.read_csv('house_test.csv')

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)

Заполнить "No_нзв_столбца":

Alley,
Fence,
FireplaceQu,
PoolQC,
MiscFeature,
GarageCond,
GarageType,
GarageFinish,
GarageQual,
BsmtExposure,
BsmtFinType2,
BsmtFinType1,
BsmtCond, 
BsmtQual,
Electrical,
MSZoning,
Utilities,
Functional,
KitchenQual,
SaleType,
Exterior2nd

Заполнить None:
MasVnrType

Заполнить mode:
GarageYrBlt,
BsmtFullBath,
BsmtHalfBath

Заполнить mean:
LotFrontage,
MasVnrArea,
GarageArea,
GarageCars,
BsmtFinSF2,
BsmtFinSF1,
TotalBsmtSF,
BsmtUnfSF

In [ ]:
all_data['GarageYrBlt'].mode()[0]

2005.0

In [ ]:
for column in ['LotFrontage', 'MasVnrArea', 'GarageArea', 'GarageCars', 'BsmtFinSF2', 'BsmtFinSF1','TotalBsmtSF', 'BsmtUnfSF']:
  all_data[column] = all_data[column].fillna(all_data[column].mean())

for column in ['GarageYrBlt', 'BsmtFullBath', 'BsmtHalfBath']:
  all_data[column] = all_data[column].fillna(all_data[column].mode()[0])

for column in ['Alley', 'Fence', 'FireplaceQu', 'PoolQC', 'MiscFeature', 'GarageCond', 'GarageType',
       'GarageFinish', 'GarageQual', 'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'Electrical', 'MSZoning', 'Utilities', 'Functional', 'KitchenQual', 'SaleType', 'Exterior2nd']:
       all_data[column] = all_data[column].fillna('Nocol')

all_data['MasVnrType'] = all_data['MasVnrType'].fillna('None')

for col in ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
        'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
        'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
        'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
        'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
        'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
        'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
        'SaleType', 'SaleCondition']:
  tmp = all_data[col].str.get_dummies()
  tmp = tmp.add_prefix(col + '_')
  all_data = pd.concat([all_data, tmp], axis=1)
  all_data.drop(col, axis=1, inplace=True)

In [ ]:
all_data.isna().sum().sort_values(ascending=False).head(40)

In [ ]:
del_col = []
for col in all_data.columns:
  if '_Nocol' in col:
    del_col.append(col)
del_col

['MSZoning_Nocol',
 'Alley_Nocol',
 'Utilities_Nocol',
 'Exterior2nd_Nocol',
 'BsmtQual_Nocol',
 'BsmtCond_Nocol',
 'BsmtExposure_Nocol',
 'BsmtFinType1_Nocol',
 'BsmtFinType2_Nocol',
 'Electrical_Nocol',
 'KitchenQual_Nocol',
 'Functional_Nocol',
 'FireplaceQu_Nocol',
 'GarageType_Nocol',
 'GarageFinish_Nocol',
 'GarageQual_Nocol',
 'GarageCond_Nocol',
 'PoolQC_Nocol',
 'Fence_Nocol',
 'MiscFeature_Nocol',
 'SaleType_Nocol']

In [ ]:
all_data.drop(columns='MasVnrType_None', axis=1, inplace=True)
all_data.drop(columns=del_col, axis=1, inplace=True)

In [ ]:
train_X = all_data[:1460].drop('SalePrice', axis=1)
test_X = all_data[1460:].drop('SalePrice', axis=1)
train_y = all_data[:1460]['SalePrice']

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(train_X)
train_X=scaler.transform(train_X)

xtrain, xtest, ytrain, ytest = train_test_split(train_X, train_y, test_size=0.3, random_state=0)

rfr = RandomForestRegressor()

rfr.fit(xtrain, ytrain)

pred = rfr.predict(xtest)

mean_squared_log_error(pred, ytest)

0.019367561880027386

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(train_X, train_y, test_size=0.25, random_state=0)

gbr = GradientBoostingRegressor()

gbr.fit(xtrain, ytrain)

pred = gbr.predict(xtest)

mean_squared_error(np.log(pred), np.log(ytest))

0.01630270995944077

In [ ]:
#улучшенный градиентный бустинг
xtrain, xtest, ytrain, ytest = train_test_split(train_X, train_y, test_size=0.25, random_state=42)


params={ 'learning_rate'    : [0.05, 0.10, 0.15],
         'max_depth'        : [3, 4, 5, 6, 8, 10, 12, 15],
         'n_estimators'     : [100, 500, 1000]}

gbr = GradientBoostingRegressor()

rand_search_gbr = RandomizedSearchCV(gbr, params, n_jobs=-1, cv=5)

rand_search_gbr.fit(xtrain, ytrain)

best_gbr = rand_search_gbr.best_estimator_
pred1 = best_gbr.predict(xtest)

mean_squared_log_error(ytest, pred1)

0.018153639174501125

In [ ]:
answers = pd.Series(gbr.predict(test_X))
answers_data = pd.DataFrame(test_X.Id)
answers_data['SalePrice'] = answers.values
sample = answers_data.to_csv('sample.csv',index=False)

In [ ]:
pd.Series(gbr.predict(test_X))

0       358984.604402
1       388468.811246
2       416923.569585
3       457022.478446
4       397063.878871
            ...      
1454    405019.349029
1455    387407.999654
1456    414586.402797
1457    356582.018848
1458    514280.585966
Length: 1459, dtype: float64

**Решение 2**

In [ ]:
from sklearn.preprocessing import LabelEncoder
from scipy import stats

In [ ]:
col_drop = []
for col in test.columns:
    if (train[col].isnull().sum() > 45) or (test[col].isnull().sum() > 45):
        col_drop.append(col)
train.drop(col_drop, axis=1, inplace=True)
test.drop(col_drop, axis=1, inplace=True)

In [ ]:
for col in test.columns:
    if train[col].unique().shape[0] <= 30:
        train[col].fillna(train[col].mode()[0], inplace=True)
        test[col].fillna(test[col].mode()[0], inplace=True)
        
        train[col]= LabelEncoder().fit_transform(train[col]) 
        test[col]= LabelEncoder().fit_transform(test[col]) 
        #print(col, df_train[col].unique().shape[0])
for col in test.columns:
    if (train[col].isnull().sum() > 0) or (test[col].isnull().sum() > 0):
        train[col].fillna(train[col].median(), inplace=True)
        test[col].fillna(test[col].median(), inplace=True)

In [ ]:
train.shape

(1460, 70)

In [ ]:
for col in train.columns:
    pearson_coef, p_value = stats.pearsonr(train[col], train['SalePrice'])
    print(col, pearson_coef, p_value)

In [ ]:
col_continues = []
for col in test.columns:
    if train[col].unique().shape[0] >= (train.shape[0])//10:
        col_continues.append(col)
print(col_continues)

col_skew = train[col_continues].apply(lambda x: stats.skew(x)).sort_values(ascending=False)
high_skew = col_skew[abs(col_skew) > 0.5]
high_skew

['Id', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF']


LotArea        12.195142
MasVnrArea      2.674865
OpenPorchSF     2.361912
BsmtFinSF1      1.683771
WoodDeckSF      1.539792
TotalBsmtSF     1.522688
1stFlrSF        1.375342
GrLivArea       1.365156
BsmtUnfSF       0.919323
2ndFlrSF        0.812194
dtype: float64

In [ ]:
for col in high_skew.index:
    train[col] = np.log1p(train[col])
    test[col] = np.log1p(test[col])

In [ ]:
X=train.drop('SalePrice', axis=1)
y = train.SalePrice

X = StandardScaler().fit(X).transform(X)

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=4)

model = GradientBoostingRegressor()
params={ 'learning_rate'    : [0.05, 0.10, 0.15],
         'max_depth'        : [3, 4, 5, 6, 8, 10, 12, 15],
         'n_estimators'     : [100, 500, 1000]}

random_search = RandomizedSearchCV(model, params, cv=5)
random_search.fit(xtrain, ytrain)
print(random_search.best_estimator_)
print(random_search.best_params_)
print(random_search.best_score_)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.15, loss='ls', max_depth=5,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=500,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
{'n_estimators': 500, 'max_depth': 5, 'learning_rate': 0.15}
0.8494343546354932


In [ ]:
answers_data = pd.DataFrame(test.Id)
test = StandardScaler().fit(test).transform(test)

answers = pd.Series(gbr.predict(test))
answers_data['SalePrice'] = answers.values
sample = answers_data.to_csv('sample.csv',index=False)

In [ ]:
answers

0       135693.382015
1       175185.531621
2       196907.283781
3       196886.249057
4       216243.956481
            ...      
1454     82284.759312
1455     82033.103602
1456    164647.978017
1457    126100.230361
1458    241600.826708
Length: 1459, dtype: float64